In [1]:
# Import necessary libraries
import pandas as pd
import numpy as np
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
from finta import TA
import tensorflow as tf
from tensorflow.keras.models import Sequential
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from keras.regularizers import l2
from sklearn.model_selection import KFold

In [2]:
# Load data
META_path = Path("../files/META.csv")
VIX_path = Path("../files/^VIX.csv")
FEDFUNDS_path = Path("../files/FEDFUNDS (1).csv")
SPY_index = Path("../files/SPY.csv")

In [3]:
meta_df = pd.read_csv(META_path, index_col="Date")
fear_index_df = pd.read_csv(VIX_path, index_col="Date").rename(columns={"Close": "Fear_index"})
spy_index_df = pd.read_csv(SPY_index, index_col="Date").rename(columns={"Close": "SPY_index"})
fedfunds_df = pd.read_csv(FEDFUNDS_path, index_col="DATE")

In [4]:
# Convert index to datetime index
fedfunds_df.index = pd.to_datetime(fedfunds_df.index)

# Resample the fed_funds_df to have daily frequency and forward fill the values
fedfunds_df_monthly = fedfunds_df.resample('D').ffill()


In [5]:
# Concatenate dataframes
concatenated_df = pd.concat([meta_df, fear_index_df['Fear_index'], spy_index_df['SPY_index']], axis=1)
concatenated_df.index = pd.to_datetime(concatenated_df.index)


In [6]:
# Merge with fedfunds_df
concatenated_df = pd.merge(concatenated_df, fedfunds_df_monthly, left_index=True, right_index=True)


In [7]:
# Drop rows with NaN values
concatenated_df = concatenated_df.dropna()


In [8]:
# Shift target variable
concatenated_df['Target'] = concatenated_df['Close'].shift(-5)
concatenated_df = concatenated_df.dropna()

In [9]:
concatenated_df.head(10)

,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target
2014-04-30,57.580002,59.849998,57.160000,59.779999,59.716640,76093000.0,13.41,188.309998,0.09,57.389999
2014-05-01,60.430000,62.279999,60.209999,61.150002,61.085190,82429000.0,13.25,188.330002,0.09,56.759998
2014-05-02,61.299999,61.889999,60.180000,60.459999,60.395920,54189000.0,12.91,188.059998,0.09,57.240002
2014-05-05,59.669998,61.349998,59.180000,61.220001,61.155113,46057000.0,13.29,188.419998,0.09,59.830002
2014-05-06,60.980000,61.150002,58.490002,58.529999,58.467964,55901000.0,13.80,186.779999,0.09,59.830002
2014-05-07,58.770000,59.299999,56.259998,57.389999,57.329174,78587000.0,13.40,187.880005,0.09,59.230000
2014-05-08,57.230000,58.820000,56.500000,56.759998,56.699841,61251000.0,13.43,187.679993,0.09,57.919998
2014-05-09,56.849998,57.650002,56.380001,57.240002,57.179333,52584000.0,12.92,187.960007,0.09,58.020000
2014-05-12,57.980000,59.900002,57.980000,59.830002,59.766590,48575000.0,12.23,189.789993,0.09,59.209999
2014-05-13,59.660000,60.889999,59.509998,59.830002,59.766590,48525000.0,12.13,189.960007,0.09,58.560001


In [10]:
# Calculate technical indicators using finta
data = concatenated_df.copy()  # Use the existing DataFrame concatenated_df
data['MA'] = TA.SMA(data, 20)  # 20-period Simple Moving Average
data['RSI'] = TA.RSI(data, 14)  # 14-period RSI

# Calculate Bollinger Bands correctly
bb_bands = TA.BBANDS(data, 20, 2)

# Assign Bollinger Bands values to DataFrame columns
data['BB_UPPER'] = bb_bands['BB_UPPER']
data['BB_MIDDLE'] = bb_bands['BB_MIDDLE']
data['BB_LOWER'] = bb_bands['BB_LOWER']

# Convert index to datetime
data.index = pd.to_datetime(data.index)

# Display the calculated technical indicators
data.tail()


,Open,High,Low,Close,Adj Close,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2024-03-18,491.910004,497.420013,486.809998,496.980011,496.980011,11755300.0,14.33,512.859985,5.33,503.019989,490.481999,58.321319,512.153124,490.481999,468.810874
2024-03-19,488.170013,496.630005,481.279999,496.239990,496.239990,10903100.0,13.82,515.710022,5.33,495.890015,491.706499,57.950491,511.617128,491.706499,471.795869
2024-03-20,499.500000,508.200012,495.170013,505.519989,505.519989,11711100.0,13.04,520.479980,5.33,493.859985,493.580998,61.275686,511.010667,493.580998,476.151329
2024-03-21,514.710022,515.039978,506.010010,507.760010,507.760010,9712500.0,12.92,522.200012,5.33,485.579987,494.662498,62.055680,512.814794,494.662498,476.510203
2024-03-22,507.000000,509.970001,504.339996,509.579987,509.579987,8117000.0,13.06,521.210022,5.33,491.350006,495.939998,62.712832,514.532549,495.939998,477.347447


In [11]:
# Define features and target
X = concatenated_df.drop("Close", axis=1)
y = concatenated_df["Close"]

In [12]:
data.drop(columns=['Open', 'High', 'Low', 'Close', 'Adj Close'], inplace=True)


In [13]:
# Display the modified DataFrame
data.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
2014-04-30,76093000.0,13.41,188.309998,0.09,57.389999,NaN,NaN,NaN,NaN,NaN
2014-05-01,82429000.0,13.25,188.330002,0.09,56.759998,NaN,100.000000,NaN,NaN,NaN
2014-05-02,54189000.0,12.91,188.059998,0.09,57.240002,NaN,64.834316,NaN,NaN,NaN
2014-05-05,46057000.0,13.29,188.419998,0.09,59.830002,NaN,75.185214,NaN,NaN,NaN
2014-05-06,55901000.0,13.80,186.779999,0.09,59.830002,NaN,35.431787,NaN,NaN,NaN


In [14]:
data_clean = data.dropna()
data_clean.index.rename('date', inplace=True)
data_clean.to_csv('../clean_data/META_prepared_data.csv', index=True)

In [15]:
data_clean.head()

,Volume,Fear_index,SPY_index,FEDFUNDS,Target,MA,RSI,BB_UPPER,BB_MIDDLE,BB_LOWER
date,,,,,,,,,,
2014-05-28,47795000.0,11.68,191.380005,0.09,63.340000,59.7240,68.405903,63.479537,59.7240,55.968463
2014-05-29,42700000.0,11.57,192.369995,0.09,63.189999,59.9265,69.443644,64.107416,59.9265,55.745584
2014-05-30,45253500.0,11.40,192.679993,0.09,62.500000,60.0340,65.600407,64.451255,60.0340,55.616745
2014-06-02,35996000.0,11.58,192.899994,0.10,62.880001,60.1650,64.016678,64.786144,60.1650,55.543856
2014-06-03,32217000.0,11.87,192.800003,0.10,65.769997,60.2475,62.466384,65.004854,60.2475,55.490146


In [16]:
# Define date cutoff for data split
date_cutoff = "2022-04-30"

# Split data
X_train = X[X.index <= date_cutoff]
X_test = X[X.index > date_cutoff]
y_train = y[y.index <= date_cutoff]
y_test = y[y.index > date_cutoff]

In [17]:
# Scale data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [18]:
# Define the number of folds
k = 10

# Initialize lists to store R-squared scores
train_r2_scores = []
test_r2_scores = []

# Initialize KFold
kf = KFold(n_splits=k, shuffle=True)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_train_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Example: Train with a smaller learning rate
from keras.optimizers import Adam
adam = Adam(learning_rate=0.0001)  # Adjust learning rate as needed
model.compile(optimizer=adam, loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, batch_size=32, verbose=0)

# Compile the model
model.compile(optimizer=adam, loss='mean_squared_error')

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    
    # Evaluate the model on training data
    train_predictions = model.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_r2_scores.append(train_r2)
    
    # Evaluate the model on test data
    test_predictions = model.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_r2_scores.append(test_r2)

# Calculate average R-squared scores
avg_train_r2 = np.mean(train_r2_scores)
avg_test_r2 = np.mean(test_r2_scores)

print("Average R-squared (Train):", avg_train_r2)
print("Average R-squared (Test):", avg_test_r2)

/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 576us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 828us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 355us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 565us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 370us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 390us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 513us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 338us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 369us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 533us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 370us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 539us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 339us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 521us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 386us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 485us/step
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 358us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 507us/step
Average R-squared (Train): -4.938114186766835
Average R-squared (Test): -5.026772554107444


In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Initialize lists to store metrics for selected models
selected_train_r2 = []
selected_test_r2 = []
selected_train_mae = []
selected_train_mse = []
selected_test_mae = []
selected_test_mse = []

# Perform k-fold cross-validation
for train_index, test_index in kf.split(X_train_scaled):
    X_train_cv, X_test_cv = X_train_scaled[train_index], X_train_scaled[test_index]
    y_train_cv, y_test_cv = y_train[train_index], y_train[test_index]
    
    # Define a new model for each fold
    model_fold = Sequential([
        Dense(32, activation='relu', input_shape=(X_train_cv.shape[1],)),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(16, activation='relu'),
        Dropout(0.5),  # Dropout layer with a dropout rate of 0.5
        Dense(1)  # Output layer
    ])
    
    # Compile the model with Adam optimizer and mean squared error loss
    model_fold.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')
    
    # Train the model
    model_fold.fit(X_train_cv, y_train_cv, epochs=50, batch_size=32, verbose=0)
    
    # Evaluate the model on training data
    train_predictions = model_fold.predict(X_train_cv)
    train_r2 = r2_score(y_train_cv, train_predictions)
    train_mae = mean_absolute_error(y_train_cv, train_predictions)
    train_mse = mean_squared_error(y_train_cv, train_predictions)
    
    # Evaluate the model on test data
    test_predictions = model_fold.predict(X_test_cv)
    test_r2 = r2_score(y_test_cv, test_predictions)
    test_mae = mean_absolute_error(y_test_cv, test_predictions)
    test_mse = mean_squared_error(y_test_cv, test_predictions)
    
    # Append metrics to the lists
    selected_train_r2.append(train_r2)
    selected_test_r2.append(test_r2)
    selected_train_mae.append(train_mae)
    selected_train_mse.append(train_mse)
    selected_test_mae.append(test_mae)
    selected_test_mse.append(test_mse)

# Print metrics for selected models
for idx, (train_r2, test_r2, train_mae, train_mse, test_mae, test_mse) in enumerate(zip(selected_train_r2, selected_test_r2, selected_train_mae, selected_train_mse, selected_test_mae, selected_test_mse), start=1):
    print(f"Model {idx} - Train R-squared: {train_r2}, Test R-squared: {test_r2}, Train MAE: {train_mae}, Train MSE: {train_mse}, Test MAE: {test_mae}, Test MSE: {test_mse}")


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 602us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 582us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 496us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


...

57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 663us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 525us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 694us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 514us/step


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 608us/step
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 568us/step
Model 1 - Train R-squared: 0.8769310379516377, Test R-squared: 0.885301878973993, Train MAE: 24.811057914230197, Train MSE: 745.8128141800286, Test MAE: 25.966108471951628, Test MSE: 818.2729682003719
Model 2 - Train R-squared: 0.8342111330512477, Test R-squared: 0.836674549118445, Train MAE: 28.404942422685078, Train MSE: 1028.1250963974662, Test MAE: 27.03012149718755, Test MSE: 957.4510063003282
Model 3 - Train R-squared: 0.90935764112044, Test R-squared: 0.8734334694212852, Train MAE: 21.439295363572494, Train MSE: 571.148364115367, Test MAE: 22.445974953033147, Test MSE: 627.5428837559352
Model 4 - Train R-squared: 0.9133911361223946, Test R-squared: 0.9106347653033359, Train MAE: 20.60768333549215, Train MSE: 534.735869801882, Test MAE: 21.19981073193601, Test MSE: 548.5410996626371
Model 5 - Train R-squared: 0.8701335797503598, Test R-squared: 0.8805461248823038, Train MAE: 25.172202811332053, Train 

In [20]:
# Filter models where both Train R-squared and Test R-squared are less than 0.96
filtered_indices = [i for i, (train_r2, test_r2) in enumerate(zip(selected_train_r2, selected_test_r2)) if train_r2 < 0.96 and test_r2 < 0.96]

# Calculate the absolute difference between train R-squared and test R-squared values for filtered models
abs_diff_r2_filtered = np.abs(np.array(selected_train_r2)[filtered_indices] - np.array(selected_test_r2)[filtered_indices])

# Find the index of the model with the smallest absolute difference among filtered models
best_model_index = filtered_indices[np.argmin(abs_diff_r2_filtered)]

# Retrieve the metrics for the best model
best_train_r2 = selected_train_r2[best_model_index]
best_test_r2 = selected_test_r2[best_model_index]
best_train_mae = selected_train_mae[best_model_index]
best_train_mse = selected_train_mse[best_model_index]
best_test_mae = selected_test_mae[best_model_index]
best_test_mse = selected_test_mse[best_model_index]

# Print metrics for the best model
print(f"Best Model - Train R-squared: {best_train_r2}, Test R-squared: {best_test_r2}, Train MAE: {best_train_mae}, Train MSE: {best_train_mse}, Test MAE: {best_test_mae}, Test MSE: {best_test_mse}")


Best Model - Train R-squared: 0.9057463124772043, Test R-squared: 0.9048279898189147, Train MAE: 21.5693510023611, Train MSE: 581.1444882845861, Test MAE: 21.503828541270696, Test MSE: 595.1091184966203


In [21]:
# Scale data
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Define the model architecture
model = Sequential([
    Dense(units=16, activation='relu', kernel_regularizer=l2(0.0005), input_shape=(X_scaled.shape[1],)),
    Dropout(0.6),
    Dense(units=8, activation='relu', kernel_regularizer=l2(0.0005)),
    Dropout(0.6),
    Dense(units=1)
])

# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

# Train the model on the entire dataset
model.fit(X_scaled, y, epochs=50, batch_size=32, verbose=0)

# Predict sequentially on each data point
all_predictions = model.predict(X_scaled)

# Ensure the number of predictions matches the original dataset
assert len(all_predictions) == len(X_scaled)

# Create a DataFrame to store the actual and predicted values
predictions_df = pd.DataFrame({'Actual': y, 'Predicted': all_predictions.flatten()}, index=X.index)

# Ensure index uniqueness in both the original dataset and predictions DataFrame
data_clean_unique_index = data_clean.index.drop_duplicates()
predictions_df = predictions_df.loc[data_clean_unique_index]

# Display the DataFrame
predictions_df


/Users/kd17/anaconda3/lib/python3.11/site-packages/keras/src/layers/core/dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


78/78 ━━━━━━━━━━━━━━━━━━━━ 0s 519us/step


,Actual,Predicted
date,,
2014-05-28,63.509998,55.932026
2014-05-29,63.830002,55.937088
2014-05-30,63.299999,55.723701
2014-06-02,63.080002,57.775307
2014-06-03,62.869999,59.132324
...,...,...
2024-03-18,496.980011,375.981262
2024-03-19,496.239990,374.699249
2024-03-20,505.519989,380.589722


In [22]:
predictions_df.to_csv('../predicted_data/META_predicted_data.csv', index=True)

In [23]:
# Check for NaN values in the predictions DataFrame
nan_values = predictions_df.isnull().sum().sum()

if nan_values == 0:
    print("No NaN values found in the predictions DataFrame.")
    print(predictions_df)
else:
    print(f"Found {nan_values} NaN values in the predictions DataFrame. Please check your data or model.")


No NaN values found in the predictions DataFrame.
                Actual   Predicted
date                              
2014-05-28   63.509998   55.932026
2014-05-29   63.830002   55.937088
2014-05-30   63.299999   55.723701
2014-06-02   63.080002   57.775307
2014-06-03   62.869999   59.132324
...                ...         ...
2024-03-18  496.980011  375.981262
2024-03-19  496.239990  374.699249
2024-03-20  505.519989  380.589722
2024-03-21  507.760010  387.057434
2024-03-22  509.579987  386.689667

[2473 rows x 2 columns]


In [24]:
# Calculate the percentage difference between actual and predicted values
predictions_df['Percentage Difference (%)'] = ((predictions_df['Predicted'] - predictions_df['Actual']) / predictions_df['Actual']) * 100

# Display the DataFrame with percentage difference
predictions_df


,Actual,Predicted,Percentage Difference (%)
date,,,
2014-05-28,63.509998,55.932026,-11.931936
2014-05-29,63.830002,55.937088,-12.365524
2014-05-30,63.299999,55.723701,-11.968875
2014-06-02,63.080002,57.775307,-8.409472
2014-06-03,62.869999,59.132324,-5.945085
...,...,...,...
2024-03-18,496.980011,375.981262,-24.346804
2024-03-19,496.239990,374.699249,-24.492331
2024-03-20,505.519989,380.589722,-24.713220


In [25]:
# Calculate the absolute percentage difference for each data point
predictions_df['Abs_Percentage_Diff'] = abs((predictions_df['Actual'] - predictions_df['Predicted']) / predictions_df['Actual']) * 100

# Calculate the average percentage difference
avg_percentage_diff = predictions_df['Abs_Percentage_Diff'].mean()

print("Average Percentage Difference (%):", avg_percentage_diff)


Average Percentage Difference (%): 24.875792779886833
